### Libraries

In [1]:
import pandas as pd#data structures and operations for manipulating numerical tables and time series.
import pandas_datareader.data as data#Up-to-date remote data access for pandas.
import datetime as dt#supplies classes for manipulating dates and times
from datetime import timedelta
import matplotlib.pyplot as plt#plotting library for the Python.
import seaborn as sns;# visualizacion
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings
from sklearn.preprocessing import StandardScaler
import statsmodels.tsa.api as smtsa
from statsmodels.tsa.stattools import arma_order_select_ic
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults
import itertools
import plotly.graph_objects as go

#### A.- EDP Renováveis (EDP.BE)

In [2]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
EDPR= pd.read_csv('EDPR.csv',index_col='Date',parse_dates=['Date'],date_parser=dateparse, dayfirst=True)
EDPR.head(2)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,7.1351,7.1351,7.1351,7.1351,0.0,6.866566
2016-01-05,7.3006,7.3006,7.3006,7.3006,150.0,7.025838


In [3]:
# Divide into train and validation set to calculate R-squared score and mean absolute percentage error 
train_A = EDPR[:int(0.85 * (len(EDPR)))]
test_A  = EDPR[int(0.85 * (len(EDPR))):]
train_A.shape, test_A.shape

((1973, 6), (349, 6))

ARMA

In [4]:
import warnings
warnings.filterwarnings('ignore')
arma_order_select_ic(EDPR['Close'])

{'bic':               0             1            2
 0  14617.168665  11523.098611  8918.282468
 1   -169.377693   -168.252270  -163.480339
 2   -167.742564   -161.979701  -157.329446
 3   -163.848896   -157.187564  -149.582374
 4   -156.516883   -150.443656  -157.127868,
 'bic_min_order': (1, 0)}

ARIMA

In [5]:
#defining sarima parameters

#p=is the number of autoregressive terms
#q=is the number of lagged forecast errors in the prediction equation.
#d= is the number of nonseasonal differences needed for stationarity,
aicVal=[]
for d in range(0,3):
    for ari in range(0, 3):
        for maj in range(0,3):
            try:
                arima_obj = ARIMA(EDPR['Close'].tolist(), order=(ari,d,maj))
                arima_obj_fit=arima_obj.fit()
                aicVal.append([ari, d, maj, arima_obj_fit.aic])
            except ValueError:
                pass
print(aicVal)           

param_df= pd.DataFrame(aicVal, columns=['d', 'p' ,'q', 'AIC' ])
param_df['AIC'] = param_df['AIC'].apply(pd.to_numeric, errors = 'coerce')
param_df.loc[param_df['AIC'].idxmin()]

[[0, 0, 0, 14605.66829687165], [0, 0, 1, 11505.848058142019], [1, 0, 0, -186.62824537428514], [1, 0, 1, -191.25300625798081], [1, 0, 2, -192.2312596071206], [2, 0, 0, -190.7433004134282], [2, 0, 1, -190.7306217989153], [2, 0, 2, -191.83055063765005], [0, 1, 0, -195.75166709182577], [0, 1, 1, -200.14401618375814], [0, 1, 2, -201.3062358475463], [1, 1, 0, -199.63931439425778], [1, 1, 1, -199.67151983236727], [1, 1, 2, -201.1867967587641], [2, 1, 0, -201.6930380042686], [2, 1, 1, -201.05333689069266], [2, 1, 2, -199.188482309547], [0, 2, 0, 1292.9388570758292], [0, 2, 1, -185.44283118813473], [0, 2, 2, -189.88852864631372], [1, 2, 0, 762.5109684242279], [1, 2, 1, -189.38093111284888], [1, 2, 2, -182.7577842855726], [2, 2, 0, 495.76014654660685], [2, 2, 1, -191.05556558884746], [2, 2, 2, -188.92498666595202]]


d        2.000000
p        1.000000
q        0.000000
AIC   -201.693038
Name: 14, dtype: float64

SARIMA

In [6]:
from pmdarima import auto_arima
stepwise_fit = auto_arima(EDPR['Close'], start_p=1, start_q=1,
                          max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True,
                          d=None, D=1, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=5.54 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=5389.723, Time=0.51 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=620.176, Time=2.65 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=3409.543, Time=3.21 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=5407.373, Time=0.17 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=1163.910, Time=0.33 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=374.325, Time=11.28 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=19.01 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=8.73 sec
 ARIMA(0,0,0)(2,1,0)[12] intercept   : AIC=5354.242, Time=2.52 sec
 ARIMA(2,0,0)(2,1,0)[12] intercept   : AIC=370.143, Time=12.73 sec
 ARIMA(2,0,0)(1,1,0)[12] intercept   : AIC=612.255, Time=4.08 sec
 ARIMA(2,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=29.81 sec
 ARIMA(2,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=14.35 sec
 ARIMA(3,0,0)(2,1,0)[12] intercept   : AIC=

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 2322
Model:             SARIMAX(1, 0, 1)x(2, 1, [], 12)   Log Likelihood                -179.291
Date:                             Sun, 15 May 2022   AIC                            368.581
Time:                                     14:28:24   BIC                            397.306
Sample:                                          0   HQIC                           379.052
                                            - 2322                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9646      0.003    316.477      0.000       0.959       0.971
ma.L1          0.0544      0.013      4.104      0.000       0.028       0.080
ar.S.L12      -0.6317      0.011    -56.874      0.000      -0.654      -0.610
ar.S.L24      -0.3235      0.009    -37.499      0.000      -0.340      -0.307
sigma2         0.0682      0.001     87.960      0.000       0.067       0.070
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             15835.54
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):              33.59   Skew:                             0.04
Prob(H) (two-sided):                  0.00   Kurtosis:                        15.83
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

#### B.- Vestas Wind Systems A/S (VWSB.BE)

In [7]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
VWS= pd.read_csv('VWS.csv',index_col='Date', parse_dates=['Date'],date_parser=dateparse, dayfirst=True)
VWS.head(2)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,483.998474,475.468903,473.500000,475.468903,2528.0,464.095367
2016-01-05,476.799988,465.432190,479.899994,475.200012,1738.0,463.832947


In [8]:
# Divide into train and validation set to calculate R-squared score and mean absolute percentage error 
train_B = VWS[:int(0.85 * (len(VWS)))]
test_B  = VWS[int(0.85 * (len(VWS))):]

train_B.shape, test_B.shape

((1411, 6), (249, 6))

ARMA

In [9]:
arma_order_select_ic(VWS['Close'])

{'bic':               0             1             2
 0  21470.829305  19383.328896  17888.041509
 1  13656.938374  13663.801771  13671.195976
 2  13663.797937  13670.535136  13677.936100
 3  13671.191381  13678.530949  13685.350741
 4  13678.288349  13684.354225  13680.758463,
 'bic_min_order': (1, 0)}

ARIMA

In [11]:
#defining sarima parameters

#p=is the number of autoregressive terms
#q=is the number of lagged forecast errors in the prediction equation.
#d= is the number of nonseasonal differences needed for stationarity,
aicVal=[]
for d in range(0,3):
    for ari in range(0, 3):
        for maj in range(0,3):
            try:
                arima_obj = ARIMA(VWS['Close'].tolist(), order=(ari,d,maj))
                arima_obj_fit=arima_obj.fit()
                aicVal.append([ari, d, maj, arima_obj_fit.aic])
            except ValueError:
                pass
print(aicVal)           

param_df= pd.DataFrame(aicVal, columns=['d', 'p' ,'q', 'AIC' ])
param_df['AIC'] = param_df['AIC'].apply(pd.to_numeric, errors = 'coerce')
param_df.loc[param_df['AIC'].idxmin()]

[[0, 0, 0, 21460.000159409723], [0, 0, 1, 19367.08517740713], [1, 0, 0, 13640.694655033036], [1, 0, 1, 13642.143479644052], [1, 0, 2, 13644.12311142556], [2, 0, 0, 13642.139645796935], [2, 0, 1, 13643.462272072698], [2, 0, 2, 13645.448663168423], [0, 1, 0, 13628.728545913014], [0, 1, 1, 13630.311197182249], [0, 1, 2, 13632.309351191998], [1, 1, 0, 13630.310292366044], [1, 1, 1, 13632.30465423588], [1, 1, 2, 13634.304647552799], [2, 1, 0, 13632.308097605186], [2, 1, 1, 13633.819570295704], [2, 1, 2, 13635.266389773598], [0, 2, 0, 14744.210164856517], [0, 2, 1, 13630.554250960196], [0, 2, 2, 13632.117065560165], [1, 2, 0, 14285.497209697172], [1, 2, 1, 13632.115890907018], [1, 2, 2, 13633.38256289847], [2, 2, 0, 14087.915185370328], [2, 2, 1, 13634.11195775317], [2, 2, 2, 13635.381084706802]]


d          0.000000
p          1.000000
q          0.000000
AIC    13628.728546
Name: 8, dtype: float64

SARIMA

In [12]:
from pmdarima import auto_arima
stepwise_fit = auto_arima(VWS['Close'], start_p=0, start_q=0,
                          max_p=3, max_q=8, m=12,
                          start_P=0, seasonal=True,
                          d=None, D=1, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=17602.692, Time=0.86 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=17602.069, Time=0.08 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=14179.114, Time=1.58 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=16108.142, Time=2.80 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=17602.868, Time=0.08 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=14590.926, Time=0.25 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=13991.929, Time=6.67 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=7.53 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=5.66 sec
 ARIMA(0,0,0)(2,1,0)[12] intercept   : AIC=17586.970, Time=3.58 sec
 ARIMA(2,0,0)(2,1,0)[12] intercept   : AIC=13993.671, Time=11.58 sec
 ARIMA(1,0,1)(2,1,0)[12] intercept   : AIC=13993.683, Time=9.25 sec
 ARIMA(0,0,1)(2,1,0)[12] intercept   : AIC=16096.708, Time=9.42 sec
 ARIMA(2,0,1)(2,1,0)[12] intercept   : AIC=inf, Time=16.36 sec
 ARIMA(1,0,0)(2,1,0)[

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 1660
Model:             SARIMAX(1, 0, 0)x(2, 1, 0, 12)   Log Likelihood               -6991.086
Date:                            Sun, 15 May 2022   AIC                          13990.173
Time:                                    14:33:15   BIC                          14011.802
Sample:                                         0   HQIC                         13998.193
                                           - 1660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9597      0.004    271.819      0.000       0.953       0.967
ar.S.L12      -0.6398      0.004   -171.900      0.000      -0.647      -0.632
ar.S.L24      -0.3314      0.004    -92.958      0.000      -0.338      -0.324
sigma2       281.9383      0.752    375.116      0.000     280.465     283.411
===================================================================================
Ljung-Box (L1) (Q):                   0.23   Jarque-Bera (JB):           9154963.64
Prob(Q):                              0.63   Prob(JB):                         0.00
Heteroskedasticity (H):               4.15   Skew:                           -11.88
Prob(H) (two-sided):                  0.00   Kurtosis:                       367.36
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""